In [8]:
import pandas as pd
import random
from faker import Faker
import calendar

# Initialize Faker
fake = Faker()

# Define categories and payment modes
categories = ["Groceries", "Stationary", "Bills", "Subscription", "Investment", "Entertainment", "Healthcare", "Food", "Travel"]
payment_mode = ["UPI", "Cash", "Debit Card", "Credit Card"]

# Function to generate expense data for a given month
def gen_expenses_for_month(year, month, num_entries=150):
    _, last_day = calendar.monthrange(year, month)  # Get the last day of the month
    data = []
    for _ in range(num_entries):
        expense = {
            "Date": fake.date_between(start_date=pd.Timestamp(f'{year}-{month:02d}-01'),
                                      end_date=pd.Timestamp(f'{year}-{month:02d}-{last_day}')).strftime('%Y-%m-%d'),
            "Category": random.choice(categories),
            "Payment_mode": random.choice(payment_mode),
            "Description": fake.sentence(),
            "Amount": round(random.uniform(10, 1000), 2),
            "Cashback": round(fake.random_number(digits=2) / 100, 2) if random.choice(payment_mode) != 'Cash' else 0.00
        }
        data.append(expense)
    return pd.DataFrame(data)

# Generate data for each month and export to CSV
def generate_and_export_all(year, num_entries=150):
    for month in range(1, 13):  # Iterate through months 1 to 12
        df = gen_expenses_for_month(year, month, num_entries)
        filename = f"fake_expense_data_{calendar.month_abbr[month].lower()}.csv"
        df.to_csv(filename, index=False)
        print(f"Exported data for {calendar.month_name[month]} to {filename}")

# Generate and export data for 2024
generate_and_export_all(2024, 150)


Exported data for January to fake_expense_data_jan.csv
Exported data for February to fake_expense_data_feb.csv
Exported data for March to fake_expense_data_mar.csv
Exported data for April to fake_expense_data_apr.csv
Exported data for May to fake_expense_data_may.csv
Exported data for June to fake_expense_data_jun.csv
Exported data for July to fake_expense_data_jul.csv
Exported data for August to fake_expense_data_aug.csv
Exported data for September to fake_expense_data_sep.csv
Exported data for October to fake_expense_data_oct.csv
Exported data for November to fake_expense_data_nov.csv
Exported data for December to fake_expense_data_dec.csv


# Merge 12csv into single csv

In [9]:
# Merge 12csv into single csv
import pandas as pd
import os

# Folder containing the CSV files
csv_folder = "D:/Streamlit"  # Update with the path to your folder

# List to store DataFrames
dataframes = []

# Loop through all CSV files in the folder
for file in os.listdir(csv_folder):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(csv_folder, file)
        df = pd.read_csv(file_path)  # Read CSV into a DataFrame
        dataframes.append(df)  # Add DataFrame to the list

# Merge all DataFrames into one
merged_data = pd.concat(dataframes, ignore_index=True)

# Save the merged DataFrame to a new CSV
merged_data.to_csv("merged_expense_data.csv", index=False)

print("Merged CSV file has been saved as 'merged_expense_data.csv'")


Merged CSV file has been saved as 'merged_expense_data.csv'


# Import CSV data into MYSQL

In [10]:
# Import CSV data into MYSQL
import pandas as pd
import pymysql

# Connect to MySQL Database
conn = pymysql.connect(
    host="localhost",
    user="root",          # or your MySQL username
    password="12345678",  # your MySQL password
    database="EXPENSE_DB" , # your MySQL database name
    connect_timeout=300 
)

cursor = conn.cursor()

# Read CSV file into DataFrame
csv_file_path = 'D:/Streamlit/merged_expense_data.csv'
df = pd.read_csv(csv_file_path)

# Format Amount to 2 decimal places
df['Amount'] = df['Amount'].fillna(0).map('{:.2f}'.format)

# Ensure date format conversion to the correct format (auto-infer datetime format)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True).dt.strftime('%y-%m-%d')

# Select the database
conn.select_db("EXPENSE_DB")

# Create the table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS expense_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Date DATE NOT NULL,
        Category VARCHAR(50) NOT NULL,
        Payment_mode VARCHAR(50) NOT NULL,
        Description VARCHAR(255),
        Amount DECIMAL(10, 2) NOT NULL,
        Cashback DECIMAL(10, 2) NOT NULL
    )
''')

# Insert data from CSV into the table
insert_query = '''
    INSERT INTO expense_data (Date, Category, Payment_mode, Description, Amount, Cashback)
    VALUES (%s, %s, %s, %s, %s, %s)
'''

for index, row in df.iterrows():
    cursor.execute(insert_query, (
        row['Date'],  # Use the correctly formatted Date
        row['Category'],
        row['Payment_mode'],
        row['Description'],
        row['Amount'],
        row['Cashback']  
    ))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("CSV data has been imported into the MySQL table.")

C:\Users\ElakkiyaManavalan\AppData\Local\Temp\ipykernel_12912\120314881.py:24: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True).dt.strftime('%y-%m-%d')


CSV data has been imported into the MySQL table.


In [2]:
import pymysql

# Database connection
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="12345678"
)

print("Connection successful!")

Connection successful!


In [3]:
mycursor=conn.cursor()

In [ ]:
mycursor.execute("CREATE DATABASE EXPENSE1")


In [6]:
mycursor.execute("USE EXPENSE1")

0